In [1]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random


DATA_DIR = '/home/moon/traindata/qrcode/'

width = 52
height = 24
target_size = (width, height)
LR = 0.001
n_class = 37 # 加了一个空



train_dir = os.path.join(DATA_DIR, 'train')

validation_dir = os.path.join(DATA_DIR, 'test')


In [8]:
# image generator
import string
characters = string.digits + string.ascii_uppercase

def generate_from_path(path, batch_size=10):
    def _convert_str2label(s):
        s = s.split('_')[1].replace('.png', '')
        total_length = 4
        s_length = len(s)
        assert len(s) <= total_length
        result = []
        for i in range(total_length): 
            if i < s_length:
                result.append(characters.find(s[i].upper()))
            else:
                result.append(36)
#         matrix = np.zeros((total_length, n_class), dtype=np.uint8)
#         for i, index in enumerate(result):
#             matrix[i][index] = 1
#         return matrix
        return result
    
    def _parse_function(filepath, label):
        global width, height
        image_string = tf.read_file(filepath)
        image_decoded = tf.image.decode_jpeg(image_string, channels=3)
        # 图像处理，prevent overfiting
        # resize
        image_decoded = tf.image.resize_image_with_pad(image_decoded, height, width)
        image = tf.cast(image_decoded, tf.float32)

        return image, label
    
    
    
    filenames = os.listdir(path)
    filepaths = [os.path.join(path, i) for i in filenames]
    labels = [_convert_str2label(i) for i in filenames]
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    
    
    
    dataset = dataset.map(_parse_function)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.shuffle(batch_size*2)
    
#     iterator = dataset.make_one_shot_iterator()
#     iterator = dataset.make_initializable_iterator()
    
    return dataset

train_iterator = generate_from_path(train_dir)
validation_iterator = generate_from_path(validation_dir)

images, labels = train_iterator.get_next()

sess = tf.Session()
i, l = sess.run([images, labels])
print(i.shape, l.shape)


AttributeError: 'BatchDataset' object has no attribute 'get_next'

In [5]:
img_input = layers.Input((height, width,  3))

x = layers.Conv2D(16, 3, activation='relu')(img_input)
print(x.shape)
x = layers.MaxPool2D(2)(x)
print(x.shape)

x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPool2D(2)(x)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPool2D(2)(x)

x = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

output = [layers.Dense(4, activation='softmax', name='c%d'%(i+1))(x) for i in range(4)]
print(output[-1].shape)

model = Model(img_input, output)
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=LR),
              metrics=['accuracy']
              )

(?, 22, 50, 16)
(?, 11, 25, 16)
(?, 4)


In [4]:
history = model.fit_generator(
    train_iterator.get_next(),
    steps_per_epoch=1000,
    epochs=30,
    validation_data=validation_iterator.get_next(),
    validation_steps=50,
    verbose=2
)

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Accuracy')

plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Loss')

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 4 array(s), but instead got the following list of 1 arrays: [<tf.Tensor 'IteratorGetNext_2:1' shape=(?, 4) dtype=int32>]...